This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

# 自然語言處理任務：文本分類

請記得先複製一份在你自己的 Google 帳號底下：
`檔案` -> `在雲端硬碟中儲存副本`

也請記得改變 colab 設定：
1. `工具` -> `設定` -> `編輯器` -> 將 `縮排寬度` 改為 `4`
2. `編輯` -> `筆記本設定` -> 將 `硬體加速器` 改為 `GPU`

## 練習題：飯店地雷偵測儀

在旅遊找飯店或旅館，大家時常會透過訂房網站預訂。在預定前，除了看飯店的平均分數之外，許多人也會習慣看過往旅客對該飯店的評論。不過評論通常會非常多，因此要用人力看完並且一一分析是非常困難的。在這個練習裡，我們試著去訓練一個文本分類的模型，分辨每則評論是正面（positive）或是負面（negative）的，並且計算該飯店所有評論的正面及負面的比例為多少，讓我們能快速判斷是否為地雷！

訓練資料來自[這裡](https://github.com/Chunshan-Theta/NLPLab/tree/master/class_model/sentiment/py3/zn)，總共從 booking.com 蒐集 341,658 則評論，我們擷取其中的 1,200 條，包含 1,000 條的訓練資料，以及 200 條的測試資料。

![](https://i.imgur.com/FjUmObT.png)
（[圖片來源](https://memes.tw/wtf/229944)）

In [ ]:
!pip install numpy pandas torch simpletransformers

In [ ]:
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel

### 資料準備

首先我們先下載訓練資料（`sent_train.csv`）與測試資料（`sent_test.csv`）到 Colab Server 的硬碟裡。

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/hotel_sentimental/sent_train.csv -O sent_train.csv

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/hotel_sentimental/sent_test.csv -O sent_test.csv

由於資料是 csv 檔，我們可以直接用 pandas 讀取。資料有兩個 column，分別是 `labels` 跟 `text`。每個 row 則代表一則評論，`text` 為評論的內容，`labels` 如果是 1 代表為正面（positive），0 則代表為負面。

In [ ]:
train_df = pd.read_csv("sent_train.csv")
test_df = pd.read_csv("sent_test.csv")

In [ ]:
print(train_df.head())
print(test_df.head())

這裡來查看一下資料的分佈狀況。

In [ ]:
print("Distribution for training data:")
print(train_df["labels"].value_counts())

In [ ]:
print("Distribution for testing data:")
print(test_df["labels"].value_counts())

### 模型訓練

這裡我們要用 [Simple Transformers](https://github.com/ThilinaRajapakse/simpletransformers) 的 [text classification](https://simpletransformers.ai/docs/classification-specifics/) 來建立模型、訓練以及測試。

模型的架構我們使用 BERT，是由 CKIP Lab 針對中文文本做過預訓練的，我們要用它來做下游任務的 fine-tuning。相關的參數請參考[這裡](https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model)。

In [ ]:
# Create a classification model based on the pre-trained BERT.
model = ClassificationModel(
    "bert",
    "ckiplab/bert-base-chinese",
    args={
        "learning_rate": 0.0001,
        "num_train_epochs": 2,
        "train_batch_size": 8,
        "overwrite_output_dir": True,
    },
)

訓練模型非常簡單，只要呼叫 `train_model()`，等待結果即可。

In [ ]:
# Train the model.
model.train_model(train_df)

做 evaluation 也非常簡單，只要呼叫 `eval_model()` 就會把對測試資料相關的 evaluation metrics 等資訊計算出來，其中各種 metrics 的代表意義請參考[這篇文章](https://medium.com/ai%E5%8F%8D%E6%96%97%E5%9F%8E/evaluation-metrics-%E5%88%86%E9%A1%9E%E6%A8%A1%E5%9E%8B-ba17ad826599)。

In [ ]:
# Evaluate the model.
result, model_outputs, wrong_predictions = model.eval_model(test_df)
print(result)

In [ ]:
num_correct = (result["tp"] + result["tn"]) 
num_total = (result["tp"] + result["tn"] + result["fp"] + result["fn"])
accuracy = num_correct / num_total

print("Accuracy: {}".format(accuracy))

如果只是要單純的拿到預測結果，呼叫 `predict()` 即可。

In [ ]:
# Make predictions with the model.
predictions, raw_outputs = model.predict(["被子不太透風有點悶，材質粗粗的😣😣😣"])
print(predictions)

In [ ]:
predictions, raw_outputs = model.predict(["喜歡他在商圈附近，地標明顯。"])
print(predictions)

### 偵測地雷

我們拿某家飯店真實的評論，擷取其中的 20 則，來看一下整體的評論如何，是否為地雷吧！

In [ ]:
hotel_reviews = [
    "每個服務人員都超級熱心親切友善❤️",
    "寵物友善旅店👍🏻👍🏻，乾淨舒適，推薦😊",
    "床不舒適，枕頭背面還有別人的頭髮",
    "床上有發現幾根頭髮， 冰箱沒有先插電所以一進房想冰東西會需等等唷！ 我們住的這間202，房間冷氣會發出奇怪的聲音，窗外有顆大樹，一早就被鳥啄窗戶叫醒（哈～） 廁所沒有插座這點比較不方便！ 旅棧的公共空間有點線香，味道實在太濃了，建議可增加通風讓口氣流通比較好唷！",
    "走道通風較不良",
    "地點方便, 早餐不錯, 已住第二次, 滿意",
    "地點好 大路邊 離駁二與渡輪站都很近 而且還有提供免費腳踏車可借",
    "好吃",
    "枕頭可以用札實一點材質",
    "早餐時段只有一人服務",
    "早餐好吃，飲料很大杯，服務超讚，還能提供接駁，讓我們來得及演唱會，而且還有幫我們留車位，服務超好，而且超多重機，佈置很棒。",
    "都很捧!",
    "cp值高",
    "浴室乾濕不分離",
    "物美價廉，值得一住",
    "早餐非常豐富好吃 房間設備蠻新的",
    "早餐尚可",
    "員工服務熱心，房間整潔",
    "主人給予很多協助 早餐好吃 員工服務周到",
    "要開車比較方便，離捷運站較遠，附近沒有超商",
]

In [ ]:
predictions, raw_outputs = model.predict(hotel_reviews)
print(predictions)

In [ ]:
pos_count = np.count_nonzero(predictions == 1)
neg_count = np.count_nonzero(predictions == 0)

print("Positive Rate: {}".format(pos_count / len(predictions)))
print("Negative Rate: {}".format(neg_count / len(predictions)))

練習時間：

我們想要改使用英文的電影評論資料：[Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) 來做訓練，跟飯店評論一樣，我們想要訓練一個模型來分類評論是正面（positive）或是負面（negative）的。請下載[訓練資料](https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_en/movie_sentimental/sent_train.csv)以及[測試資料](https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_en/movie_sentimental/sent_test.csv)，並且用 [bert-base-cased](https://huggingface.co/bert-base-cased) 作為預訓練模型。